In [1]:
import pandas as pd
import numpy as np
from path import Path
import torch.utils.data as data
from imageio import imread
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F 
import matplotlib.pyplot as plt
%matplotlib inline 
from torch.utils.data.sampler import SubsetRandomSampler

from PIL import *
import ast

In [2]:
import numpy as np

class ScaledDotProductAttention(nn.Module):
    ''' Scaled Dot-Product Attention '''
    # given query, key,value it finds the rightful weighted component of v to get the attention applied ouput
    #q,v,k- batch X length of sequence X features or encoding
    #attention sholuld be -batchX7X7
    def __init__(self, temperature, attn_dropout=0.1):
        super().__init__()
        self.temperature = temperature
        self.dropout = nn.Dropout(attn_dropout)
        self.softmax = nn.Softmax(dim=2)

    def forward(self, q, k, v, mask=None):
#         print(k.transpose(1,2).shape)

        attn = torch.bmm(q, k.transpose(1, 2)) 
#         print(attn.shape)
        attn = attn / self.temperature

        if mask is not None:
            attn = attn.masked_fill(mask, -np.inf)

        attn = self.softmax(attn)
        attn = self.dropout(attn)
#         print(str(attn.shape)+" "+str(v.shape))
        output = torch.bmm(attn, v)

        return output, attn
SDP=ScaledDotProductAttention(5)
Ss=SDP(torch.zeros(5,6,100),torch.zeros(5,6,100),torch.zeros(5,6,100))
# print(Ss[0].shape)
class MultiHeadAttention(nn.Module):
    ''' Multi-Head Attention module '''

    def __init__(self, n_head, d_model, d_k, d_v, dropout=0.1):
        super().__init__()

        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v

        self.w_qs = nn.Linear(d_model, n_head * d_k)
        self.w_ks = nn.Linear(d_model, n_head * d_k)
        self.w_vs = nn.Linear(d_model, n_head * d_v)
        nn.init.normal_(self.w_qs.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_k)))
        nn.init.normal_(self.w_ks.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_k)))
        nn.init.normal_(self.w_vs.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_v)))

        self.attention = ScaledDotProductAttention(temperature=np.power(d_k, 0.5))
        self.layer_norm = nn.LayerNorm(d_model)

        self.fc = nn.Linear(n_head * d_v, d_model)
        nn.init.xavier_normal_(self.fc.weight)

        self.dropout = nn.Dropout(dropout)


    def forward(self, q, k, v, mask=None):

        d_k, d_v, n_head = self.d_k, self.d_v, self.n_head

        sz_b, len_q, _ = q.size()
        sz_b, len_k, _ = k.size()
        sz_b, len_v, _ = v.size()
#         print(str(sz_b)+"die")

        residual = q

        q = self.w_qs(q).view(sz_b, len_q, n_head, d_k)
        k = self.w_ks(k).view(sz_b, len_k, n_head, d_k)
        v = self.w_vs(v).view(sz_b, len_v, n_head, d_v)

        q = q.permute(2, 0, 1, 3).contiguous().view(-1, len_q, d_k) # (n*b) x lq x dk
        k = k.permute(2, 0, 1, 3).contiguous().view(-1, len_k, d_k) # (n*b) x lk x dk
        v = v.permute(2, 0, 1, 3).contiguous().view(-1, len_v, d_v) # (n*b) x lv x dv
#         print("v-"+str(v.shape))
        if mask is not None:
            mask = mask.repeat(n_head, 1, 1) # (n*b) x .. x ..
        output, attn = self.attention(q, k, v, mask=mask)
#         print(q.shape,k.shape,v.shape)
        output = output.view(n_head, sz_b, len_q, d_v)
        output = output.permute(1, 2, 0, 3).contiguous().view(sz_b, len_q, -1) # b x lq x (n*dv)

        output = self.dropout(self.fc(output))
        output = self.layer_norm(output + residual)

        return output, attn
MHA=MultiHeadAttention(4,15,15,15)
op=MHA(torch.zeros(5,7,15),torch.zeros(5,7,15),torch.zeros(5,7,15))
# print(op[0].shape)
class PositionwiseFeedForward(nn.Module):
    ''' A two-feed-forward-layer module '''

    def __init__(self, d_in, d_hid, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Conv1d(d_in, d_hid, 1) # position-wise
        self.w_2 = nn.Conv1d(d_hid, d_in, 1) # position-wise
        self.layer_norm = nn.LayerNorm(d_in)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        residual = x
        output = x.transpose(1, 2)
#         print("FCC-"+str(output.shape))
#         print("FFC_out-"+str(self.w_1(output).shape))
        output = self.w_2(F.relu(self.w_1(output)))
        output = output.transpose(1, 2)
        output = self.dropout(output)
        output = self.layer_norm(output + residual)
        return output

    

In [3]:
class EncoderLayer(nn.Module):
    ''' Compose with two layers '''

    def __init__(self, d_model, d_inner, n_head, d_k, d_v, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.slf_attn = MultiHeadAttention(
            n_head, d_model, d_k, d_v, dropout=dropout)
        self.pos_ffn = PositionwiseFeedForward(d_model, d_inner, dropout=dropout)

    def forward(self, enc_input, non_pad_mask=None, slf_attn_mask=None):
        enc_output, enc_slf_attn = self.slf_attn(
            enc_input, enc_input, enc_input, mask=slf_attn_mask)
#         print("ENC_o")
#         print(enc_output.shape)


        enc_output = self.pos_ffn(enc_output)


        return enc_output, enc_slf_attn
    
XX=EncoderLayer(15,10,4,10,10)

zz=XX(torch.zeros(5,7,15))
# print("ENc")
# print(zz[0].shape)
# print("start")
class Encoder(nn.Module):
    ''' A encoder model with self attention mechanism. '''

    def __init__(self,n_modality,d_model,n_head,d_k,d_v,dropout,n_layers,d_inner=500):
        #d_model - number of features in input 100 here
        #n_head - number of heads of multihaded attention
        #d_k=d_q=  number of features in query, key
        #d_v = number of features in value whose weighted(attentioned) sum we gonna take
        

        super().__init__()
        self.n_modality=n_modality
#         self.stn=nn.ModuleList([SpatialTransformer(3, (240,240), 8) for _ in range(n_ref)])
        
        self.layer_stack = nn.ModuleList([EncoderLayer(d_model, d_inner, n_head, d_k, d_v, dropout=dropout) 
                                          for _ in range(n_layers)])
        self.em=nn.Linear(48,100)
        #self.em1=nn.Linear(225,100)
        self.fc1=nn.Linear(d_model*n_modality,300)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(300,100)
        self.fc3=nn.Linear(100,3)
#         self.fc4=nn.Linear(50,3)
        self.bn1 = nn.BatchNorm1d(num_features=300)
        self.bn2 = nn.BatchNorm1d(num_features=100)
        self.softmax=nn.Softmax(1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, embeddings1,embeddings2,embeddings3 ):#mlpxlstmx3d


        
        encodings_total=[embeddings1,embeddings2,self.em(embeddings3)]

        enc_output=torch.stack(encodings_total,0)

        
        enc_output=enc_output.permute(1,0,2)
#         print("encoding_OUTPUT2-"+str(enc_output.shape))

        for enc_layer in self.layer_stack:
            enc_output, enc_slf_attn = enc_layer(enc_output,non_pad_mask=None,slf_attn_mask=None)
           
        
        final_input=enc_output.reshape(enc_output.shape[0],-1)

        final=self.relu(self.fc3(self.bn2(self.relu((self.fc2(self.bn1(self.relu(self.fc1(final_input)))))))))
        
        return(final)
     

In [4]:
e1=torch.zeros(4,100)
e2=torch.zeros(4,100)
e3=torch.stack([e1,e2],0)
e3.shape

torch.Size([2, 4, 100])

In [5]:
feature_set_MLP=pd.read_csv('BCLL_MLP_features_ankit.csv')
feature_set_3d=pd.read_csv('AnkitBCLL_3d.csv')
feature_set_LSTM=pd.read_csv('BCLL_LSTM_features_ankit.csv')
feature_set_3d.head()

,gene_name,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_38,tag_39,tag_40,tag_41,tag_42,tag_43,tag_44,tag_45,tag_46,tag_47
0,hint1,0.000128,0.003666,0.000086,-0.000142,0.013800,0.000378,-0.000040,3.907078e-03,-0.000168,...,0.000020,-0.000014,-0.008416,0.000020,-0.000204,0.005934,-0.000014,0.000322,0.003235,0.000067
1,ywhae,-0.000080,0.000155,0.003896,0.000124,0.001357,0.003743,0.000142,-5.598767e-04,-0.002663,...,0.003278,-0.000108,-0.001078,0.002478,0.000050,0.000687,0.005183,-0.000012,0.000014,-0.001313
2,kat2b,-0.000067,0.003084,0.003566,-0.000042,0.005316,0.004484,-0.000035,8.720113e-04,-0.003647,...,0.002714,-0.000130,-0.002824,0.003957,0.000125,0.003588,0.005358,0.000133,-0.000062,0.000689
3,cib1,0.000136,0.000098,0.024427,0.000034,-0.000053,0.021380,-0.000461,1.194030e-04,-0.022567,...,0.025997,0.000306,0.000063,0.020553,0.000282,-0.000167,0.023225,-0.000222,-0.000087,-0.007981
4,cyp1a1,-0.000320,0.000653,0.009505,-0.000159,0.000983,0.010270,-0.000024,5.357184e-07,-0.007808,...,0.006218,-0.000164,-0.000459,0.007649,-0.000432,0.000244,0.009911,-0.000121,0.000433,-0.000752


In [6]:
feature_set_MLP.head()

,gene_name,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_90,tag_91,tag_92,tag_93,tag_94,tag_95,tag_96,tag_97,tag_98,tag_99
0,hint1,125133.351562,357573.156250,0.000000,587084.000000,0.0,315726.906250,0.0,175430.843750,268782.625000,...,442658.781250,0.0,465458.937500,370437.156250,0.0,0.0,472186.937500,0.0,0.0,466427.312500
1,rabggta,20101.933594,62892.046875,0.000000,103993.031250,0.0,55333.929688,0.0,30097.763672,48279.539062,...,79100.062500,0.0,82092.179688,65288.472656,0.0,0.0,83334.328125,0.0,0.0,81504.468750
2,ywhae,6624.807129,14475.721680,0.000000,24332.548828,0.0,13422.669922,0.0,9587.588867,10549.949219,...,18221.716797,0.0,19292.927734,14790.755859,0.0,0.0,19355.056641,0.0,0.0,19177.373047
3,kat2b,4142.221680,8753.067383,0.000000,14703.916016,0.0,8161.906250,0.0,5902.970215,6346.573730,...,11016.152344,0.0,11690.555664,8958.273438,0.0,0.0,11691.131836,0.0,0.0,11626.661133
4,smad5,1322.737061,2288.863037,26.059177,3835.314453,0.0,2206.577148,0.0,1791.832397,1568.654663,...,2822.048828,0.0,3072.452148,2335.262207,0.0,0.0,3047.292969,0.0,0.0,3087.742920


In [7]:
header_of_MLP=['tag_'+str(i) for i in range(feature_set_MLP.shape[1]-1)]
features_MLP=np.array(feature_set_MLP[header_of_MLP])
gene_MLP=feature_set_MLP['gene_name']
print(features_MLP.shape)
print(len(gene_MLP))

(4655, 100)
4655


In [8]:
dictionary_MLP={}
u=0
for gn in gene_MLP:
    dictionary_MLP[gn]=features_MLP[u]
    u=u+1

In [9]:
header_of_3d=['tag_'+str(i) for i in range(feature_set_3d.shape[1]-1)]
features_3d=np.array(feature_set_3d[header_of_3d])
gene_3d=feature_set_3d['gene_name']
print(features_3d.shape)
print(len(gene_3d))


(2244, 48)
2244


In [10]:
dictionary_3d={}
u=f=0
for gn in gene_3d:
    if gn in dictionary_3d.keys():
#         print(gn)
        f=f+1
    dictionary_3d[gn]=features_3d[u]
    u=u+1
print(u)

2244


In [11]:
print(len(dictionary_3d.keys()))

1935


In [12]:
header_of_lstm=['tag_'+str(i) for i in range(feature_set_LSTM.shape[1]-1)]
features_lstm=np.array(feature_set_LSTM[header_of_lstm])
gene_lstm=feature_set_LSTM['gene_name']
print(features_lstm.shape)
print(len(gene_lstm))

(4275, 100)
4275


In [13]:
dictionary_lstm={}
u=f=0
for gn in gene_lstm:
    if gn in dictionary_lstm.keys():
#         print(gn)
        f=f+1
    dictionary_lstm[gn]=features_lstm[u]
    u=u+1
print(u)

4275


In [14]:
fil=open('../../Multi-modality/Model/BCLL/uni model/MLP/label0.txt','r')
tmp=list()
for line in fil:
	tmp.append(int(line))

label_BCLL=np.array(tmp)
print(label_BCLL.shape)

(4655,)


In [21]:
class Sequenceloader(data.Dataset):
    def __init__(self,GN,label,Feat):
        self.gene_names=GN
        self.features_mlp=Feat
        self.label=label
        self.coincdgene_name=[]
        self.coincidfeature_lstm=[]
        self.coincidfeature_3d=[]
        self.coincidfeature_mlp=[]
        self.coincidlabel=[]
        for i in range(len(self.gene_names)):
            u=self.gene_names[i]
            if u in dictionary_3d.keys() and u in dictionary_lstm.keys():
                
                if np.array(self.label[i])==0:
                    ch=50
                elif np.array(self.label[i])==1:
                    ch=7
                else:
                    ch=1
                    
                for jj in range(ch):
                    self.coincdgene_name.append(u)
                    self.coincidfeature_lstm.append(dictionary_lstm[u])
                    self.coincidfeature_3d.append(dictionary_3d[u])
                    self.coincidlabel.append(self.label[i])
                    self.coincidfeature_mlp.append(self.features_mlp[i])
        
        
    
    def __len__(self):
#         print(len(self.dataset))
        return len(self.coincdgene_name)       

    def __getitem__(self, index):
         return np.array(self.coincidfeature_mlp[index]),np.array(self.coincidfeature_lstm[index]),np.array(self.coincidfeature_3d[index]),np.array(self.coincidlabel[index])
#         print(self.dataset['gen_name'][index])          
#         try:
           
#         except :
            
total_set=Sequenceloader(gene_MLP,label_BCLL,features_MLP)  
a=b=c=0
for x1,x2,x3,z in total_set:
    if(z==0):
        a=a+1
    elif z==1:
        b=b+1
    else:
        c=c+1
print(a,b,c)
    

1900 2002 1911


In [22]:
batch_size = 4
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(total_set)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(total_set, batch_size=batch_size, 
                                           sampler=train_sampler,drop_last=True)
validation_loader = torch.utils.data.DataLoader(total_set, batch_size=batch_size,
                                                sampler=valid_sampler,drop_last=True)


In [23]:
for a,b,c,d in train_loader:
    print(a.shape,b.shape,c.shape,d.shape)

torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) to

torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) to

torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) to

torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) to

torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4, 48]) to

In [24]:
def test(test_loader,model):

    total_imgs=0;
    total_corrects=0
    u=0
    nb_classes=3
    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    for i1,i2,i3,label in test_loader:
                

        output=model(i1.to(device).float(),i2.to(device).float(),i3.to(device).float())
        total_imgs=total_imgs+label.shape[0]
        z=torch.max(output,1)[1]==label.to(device)
        _, preds = torch.max(output, 1)
#         print(output.shape)
 
        num_corrects=torch.sum(z)
        total_corrects=total_corrects+num_corrects
        for t, p in zip(label.view(-1), preds.view(-1)):
            confusion_matrix[t.long(), p.long()] += 1


        u=u+1
    
    
    print(confusion_matrix)
    return(total_corrects,total_imgs)

In [26]:
device='cuda:7'
model_3dXLSTMXMLP=Encoder(3,100,4,300,300,True,4).to(device)
#model_3dXLSTMXMLP.load_state_dict(torch.load(Path('2AnkitILD COMBO_3DXLSTMXMLP.pt')))
uz=vz=torch.rand(4,100).to(device)
# model_3dXMLP(uz,vz).shape


In [27]:

# model.load_state_dict(torch.load(Path('currentz_m1_epoch.pt')))
optim_params = [
    {'params': model_3dXLSTMXMLP.parameters(), 'lr': 0.0001}
]
optimizer = torch.optim.Adam(optim_params)
criterion = nn.CrossEntropyLoss()

In [28]:
epoch=10000
for i in range(epoch):
    torch.save(model_3dXLSTMXMLP.state_dict(), '1AnkitBCLL COMBO_3DXLSTMXMLP.pt')
    print("Accuracy-"+str(test(validation_loader,model_3dXLSTMXMLP)))
    total_loss=0
    for inp1,inp2,inp3,lab in train_loader:

        

        output=model_3dXLSTMXMLP(inp1.to(device).float(),inp2.to(device).float(),inp3.to(device).float())

        loss_batch=criterion(output,lab.to(device))/4
        optimizer.zero_grad()  
        loss_batch.backward()
        optimizer.step()
        total_loss+=loss_batch
       
    print(total_loss)
    
    
        
        

tensor([[242.,  40.,  79.],
        [ 74., 123., 208.],
        [203., 154.,  37.]])
Accuracy-(tensor(402, device='cuda:7'), 1160)
tensor(119.0973, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[343.,  13.,   5.],
        [ 13., 372.,  20.],
        [  5.,  29., 360.]])
Accuracy-(tensor(1075, device='cuda:7'), 1160)
tensor(77.3425, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[338.,  19.,   4.],
        [ 15., 367.,  24.],
        [  9.,  34., 350.]])
Accuracy-(tensor(1055, device='cuda:7'), 1160)
tensor(69.1619, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[344.,  15.,   2.],
        [  6., 375.,  24.],
        [  9.,  28., 357.]])
Accuracy-(tensor(1076, device='cuda:7'), 1160)
tensor(69.4800, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[345.,  15.,   2.],
        [  7., 377.,  20.],
        [  6.,  25., 363.]])
Accuracy-(tensor(1085, device='cuda:7'), 1160)
tensor(64.6068, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[343.,  16.,   2.],
        [ 13., 371.,  21.],

tensor(44.1178, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[361.,   1.,   0.],
        [  8., 384.,  13.],
        [  5.,  27., 361.]])
Accuracy-(tensor(1106, device='cuda:7'), 1160)
tensor(48.0006, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[356.,   2.,   3.],
        [  8., 385.,  13.],
        [  5.,  28., 360.]])
Accuracy-(tensor(1101, device='cuda:7'), 1160)
tensor(47.4056, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[357.,   5.,   0.],
        [  8., 378.,  19.],
        [  6.,  25., 362.]])
Accuracy-(tensor(1097, device='cuda:7'), 1160)
tensor(44.9024, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[356.,   4.,   1.],
        [  8., 382.,  16.],
        [  4.,  29., 360.]])
Accuracy-(tensor(1098, device='cuda:7'), 1160)
tensor(44.6001, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[351.,   5.,   5.],
        [  6., 385.,  15.],
        [  8.,  26., 359.]])
Accuracy-(tensor(1095, device='cuda:7'), 1160)
tensor(41.4081, device='cuda:7', grad_fn=<AddBackward0>

tensor(39.9850, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[356.,   4.,   1.],
        [ 10., 384.,  12.],
        [  7.,  24., 362.]])
Accuracy-(tensor(1102, device='cuda:7'), 1160)
tensor(37.6646, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[357.,   4.,   0.],
        [ 10., 384.,  11.],
        [  6.,  24., 364.]])
Accuracy-(tensor(1105, device='cuda:7'), 1160)
tensor(38.4131, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[360.,   1.,   1.],
        [  9., 386.,  10.],
        [  7.,  29., 357.]])
Accuracy-(tensor(1103, device='cuda:7'), 1160)
tensor(37.8778, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[360.,   0.,   0.],
        [  6., 383.,  17.],
        [  7.,  29., 358.]])
Accuracy-(tensor(1101, device='cuda:7'), 1160)
tensor(39.5610, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[359.,   2.,   0.],
        [ 10., 390.,   5.],
        [ 11.,  26., 357.]])
Accuracy-(tensor(1106, device='cuda:7'), 1160)
tensor(36.2677, device='cuda:7', grad_fn=<AddBackward0>

tensor(33.6602, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[361.,   0.,   0.],
        [  9., 381.,  16.],
        [  7.,  24., 362.]])
Accuracy-(tensor(1104, device='cuda:7'), 1160)
tensor(36.4375, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[361.,   1.,   0.],
        [  9., 384.,  12.],
        [  6.,  25., 362.]])
Accuracy-(tensor(1107, device='cuda:7'), 1160)
tensor(34.4056, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[362.,   0.,   0.],
        [  7., 394.,   4.],
        [  6.,  27., 360.]])
Accuracy-(tensor(1116, device='cuda:7'), 1160)
tensor(33.4354, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[359.,   1.,   1.],
        [ 12., 392.,   1.],
        [  6.,  31., 357.]])
Accuracy-(tensor(1108, device='cuda:7'), 1160)
tensor(34.3048, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[360.,   1.,   1.],
        [  6., 388.,  11.],
        [ 10.,  26., 357.]])
Accuracy-(tensor(1105, device='cuda:7'), 1160)
tensor(35.0937, device='cuda:7', grad_fn=<AddBackward0>

tensor(35.9550, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[354.,   8.,   0.],
        [  7., 384.,  14.],
        [  2.,  22., 369.]])
Accuracy-(tensor(1107, device='cuda:7'), 1160)
tensor(35.5125, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[360.,   1.,   0.],
        [  5., 389.,  12.],
        [  7.,  22., 364.]])
Accuracy-(tensor(1113, device='cuda:7'), 1160)
tensor(33.6402, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[357.,   5.,   0.],
        [  6., 389.,  10.],
        [  8.,  21., 364.]])
Accuracy-(tensor(1110, device='cuda:7'), 1160)
tensor(34.6704, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[357.,   4.,   1.],
        [ 12., 383.,  11.],
        [  9.,  19., 364.]])
Accuracy-(tensor(1104, device='cuda:7'), 1160)
tensor(31.6273, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[362.,   0.,   0.],
        [ 10., 386.,   9.],
        [  6.,  25., 362.]])
Accuracy-(tensor(1110, device='cuda:7'), 1160)
tensor(36.6955, device='cuda:7', grad_fn=<AddBackward0>

tensor(31.4460, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[357.,   5.,   0.],
        [  4., 391.,  10.],
        [  3.,  21., 369.]])
Accuracy-(tensor(1117, device='cuda:7'), 1160)
tensor(33.0781, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[361.,   0.,   0.],
        [  6., 394.,   5.],
        [  8.,  24., 362.]])
Accuracy-(tensor(1117, device='cuda:7'), 1160)
tensor(33.6803, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[361.,   0.,   0.],
        [  7., 387.,  11.],
        [  8.,  21., 365.]])
Accuracy-(tensor(1113, device='cuda:7'), 1160)
tensor(30.0582, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[359.,   1.,   1.],
        [ 11., 388.,   7.],
        [  7.,  22., 364.]])
Accuracy-(tensor(1111, device='cuda:7'), 1160)
tensor(31.2117, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[360.,   1.,   0.],
        [  5., 396.,   5.],
        [  3.,  25., 365.]])
Accuracy-(tensor(1121, device='cuda:7'), 1160)
tensor(30.4915, device='cuda:7', grad_fn=<AddBackward0>

tensor(31.9464, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[359.,   3.,   0.],
        [  9., 387.,   9.],
        [  6.,  23., 364.]])
Accuracy-(tensor(1110, device='cuda:7'), 1160)
tensor(32.6740, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[361.,   1.,   0.],
        [  6., 387.,  12.],
        [  3.,  26., 364.]])
Accuracy-(tensor(1112, device='cuda:7'), 1160)
tensor(34.5522, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[361.,   0.,   0.],
        [  8., 390.,   8.],
        [  7.,  26., 360.]])
Accuracy-(tensor(1111, device='cuda:7'), 1160)
tensor(34.5154, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[360.,   1.,   0.],
        [  7., 391.,   7.],
        [  6.,  25., 363.]])
Accuracy-(tensor(1114, device='cuda:7'), 1160)
tensor(33.1443, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[358.,   3.,   0.],
        [  2., 397.,   6.],
        [  2.,  25., 367.]])
Accuracy-(tensor(1122, device='cuda:7'), 1160)
tensor(31.5241, device='cuda:7', grad_fn=<AddBackward0>

tensor(31.9286, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[359.,   2.,   0.],
        [  7., 388.,  10.],
        [  7.,  22., 365.]])
Accuracy-(tensor(1112, device='cuda:7'), 1160)
tensor(33.1850, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[359.,   2.,   0.],
        [  5., 394.,   7.],
        [  4.,  23., 366.]])
Accuracy-(tensor(1119, device='cuda:7'), 1160)
tensor(30.9649, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[359.,   3.,   0.],
        [  9., 391.,   6.],
        [  5.,  23., 364.]])
Accuracy-(tensor(1114, device='cuda:7'), 1160)
tensor(29.4225, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[361.,   1.,   0.],
        [  6., 392.,   7.],
        [  5.,  25., 363.]])
Accuracy-(tensor(1116, device='cuda:7'), 1160)
tensor(30.6137, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[362.,   0.,   0.],
        [  4., 399.,   3.],
        [  4.,  25., 363.]])
Accuracy-(tensor(1124, device='cuda:7'), 1160)
tensor(29.0942, device='cuda:7', grad_fn=<AddBackward0>

tensor(30.1539, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[362.,   0.,   0.],
        [  5., 391.,   9.],
        [  6.,  24., 363.]])
Accuracy-(tensor(1116, device='cuda:7'), 1160)
tensor(28.6491, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[359.,   3.,   0.],
        [  4., 395.,   6.],
        [  5.,  25., 363.]])
Accuracy-(tensor(1117, device='cuda:7'), 1160)
tensor(30.9317, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[356.,   4.,   1.],
        [  5., 394.,   7.],
        [  7.,  26., 360.]])
Accuracy-(tensor(1110, device='cuda:7'), 1160)
tensor(28.7418, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[360.,   1.,   0.],
        [  4., 397.,   5.],
        [  4.,  30., 359.]])
Accuracy-(tensor(1116, device='cuda:7'), 1160)
tensor(29.1297, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[357.,   3.,   1.],
        [  5., 395.,   6.],
        [  7.,  22., 364.]])
Accuracy-(tensor(1116, device='cuda:7'), 1160)
tensor(28.4037, device='cuda:7', grad_fn=<AddBackward0>

KeyboardInterrupt: 

In [26]:
!nvidia-smi
tensor([[360.,   1.,   0.],
        [  6., 394.,   5.],
        [  6.,  22., 366.]])
Accuracy-(tensor(1120, device='cuda:7'), 1160)

Thu May 30 13:48:59 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0  On |                  N/A |
| 56%   73C    P2   180W / 250W |  10666MiB / 11172MiB |     98%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 32%   56C    P2    74W / 250W |   4404MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G